In [9]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import scipy.stats


df = pd.read_csv('TuberWeights_13.07.18.csv')
df = pd.read_csv('TuberWeights_17.07.18.csv')
df_soraya = df.loc[df['Variety'] == 'Soraya'].copy()
df_venezia = df.loc[df['Variety'] == 'Venezia'].copy()

df_soraya = df_soraya[['MidSize', 'TuberWeight', 'CubeRoot']]
df_venezia = df_venezia[['MidSize', 'TuberWeight', 'CubeRoot']]

def extract_samples(df, elem='TuberWeight'):
    samples = {}
    for mid_size, tuber_weight in zip(df['MidSize'], df[elem]):
        if mid_size in samples:
            samples[mid_size].append(tuber_weight)
        else:
            samples[mid_size] = [tuber_weight]
    return samples

samples_soraya_w = extract_samples(df_soraya)
samples_soraya_cr = extract_samples(df_soraya, 'CubeRoot')

samples_venezia_w = extract_samples(df_venezia)
samples_venezia_cr = extract_samples(df_venezia, 'CubeRoot')
        
true_dist_soraya = [(x, len(samples_soraya_w[x])) for x in samples_soraya_w]
true_dist_venezia = [(x, len(samples_venezia_w[x])) for x in samples_venezia_w]


In [29]:
def create_batches(list_in, alpha=0, theta=1):
    current_index = 0
    batch_list = []
    
    assert (0 <= alpha and alpha < 1) and theta > -alpha
    for n, weight in enumerate(list_in):
        p = np.random.binomial(1, p=(theta+len(batch_list)*alpha)/(n+theta))
        if(p == 1):
            batch_list.append([weight])
        else:
            probabilities = np.asarray([len(x)-alpha for x in batch_list], dtype='float32')
            probabilities = probabilities/probabilities.sum()
            table = np.random.choice(range(len(batch_list)), 1, p=probabilities)[0]
            batch_list[table].append(weight)
    batch_list = [(len(x), np.sum(x)) for x in batch_list]
    return batch_list

# print(create_batches(samples_soraya_w[17.5], alpha=0.2))

band_to_samples_soraya_w = dict([(x, create_batches(samples_soraya_w[x], alpha=0.2)) for x in samples_soraya_w])
band_to_samples_soraya_cr = dict([(x, create_batches(samples_soraya_cr[x], alpha=0.2)) for x in samples_soraya_cr])

band_to_samples_venezia_w = dict([(x, create_batches(samples_venezia_w[x], alpha=0.2)) for x in samples_venezia_w])
band_to_samples_venezia_cr = dict([(x, create_batches(samples_venezia_cr[x], alpha=0.2)) for x in samples_venezia_cr])


In [30]:
def pi_estimate(band_to_samples, smoothing=0):
    pi_sum = {}
    num_potatoes = 0
    for band in band_to_samples:
        total_sum = 0
        for (batch_size, batch_weight) in band_to_samples[band]:
            total_sum += batch_size
        pi_sum[band] = total_sum+smoothing
        num_potatoes += total_sum+smoothing
    pi_dist = {}
    for band in pi_sum:
        pi_dist[band] = pi_sum[band]/num_potatoes
    return pi_dist

def estimate_mu(band_to_samples):
    mu = {}
    for band in band_to_samples:
        total_weight = 0
        total_potatoes = 0
        for (batch_size, batch_weight) in band_to_samples[band]:
            total_weight += batch_weight
            total_potatoes += batch_size
        mu[band] = total_weight/total_potatoes
    return mu

def estimate_var(band_to_samples, mu_estimate):
    var = {}
    for band in band_to_samples:
        square_diff_sum = 0 
        for (batch_size, batch_weight) in band_to_samples[band]:
            square_diff_sum += (batch_weight - batch_size*mu_estimate[band])**2/batch_size
        var[band] = max(square_diff_sum/len(band_to_samples[band]), 1e-2)
    return var


def get_posterior(weight, band_to_samples, pi_estimate, mu_estimate, var_estimate):
    post_prop = {}
    sum_post = 0
    post = {}
    for band in band_to_samples:
        norm_dist = scipy.stats.norm(mu_estimate[band], var_estimate[band]**(1/2))
        post_prop[band] = norm_dist.pdf(weight)*pi_estimate[band]
        sum_post += post_prop[band]

    for band in post_prop:
        post[band] = post_prop[band]/sum_post
    return post

def get_argmax(posterior):
    key_max = None
    max_value = 0.0
    for band in posterior:
        if posterior[band] > max_value:
            max_value = posterior[band]
            key_max = band
    return key_max

def kl(p, q):
    """Kullback-Leibler divergence D(P || Q) for discrete distributions
    Parameters
    ----------
    p, q : array-like, dtype=float, shape=n
    Discrete probability distributions.
    """
    p = np.asarray(p, dtype=np.float)
    q = np.asarray(q, dtype=np.float)

    return np.sum(np.where(p != 0, p * np.log(p / q), 0))

def calculate_KL(y_pred, y_true):
    y_pred = dict(y_pred)
    y_true = dict(y_true)
    
    y_pred_dist = []
    y_true_dist = []
    
    for i in y_pred:
        y_pred_dist.append(y_pred[i])
        y_true_dist.append(y_true[i])
        
    y_pred_dist = np.asarray(y_pred_dist)
    y_pred_dist = y_pred_dist/y_pred_dist.sum()
    
    y_true_dist = np.asarray(y_true_dist)
    y_true_dist = y_true_dist/y_true_dist.sum()
    
    return kl(y_true_dist, y_pred_dist)

def calculate_W(y_pred, y_true):
    y_pred = dict(y_pred)
    y_true = dict(y_true)
    
    y_pred_dist = []
    y_true_dist = []
    
    for i in y_pred:
        y_pred_dist.append(y_pred[i])
        y_true_dist.append(y_true[i])
        
    y_pred_dist = np.asarray(y_pred_dist)
    y_pred_dist = y_pred_dist/y_pred_dist.sum()
    
    y_true_dist = np.asarray(y_true_dist)
    y_true_dist = y_true_dist/y_true_dist.sum()
    
    return scipy.stats.wasserstein_distance(y_true_dist, y_pred_dist)



In [31]:
def get_pseudo_counts(band_to_samples, count_df):
    pi = pi_estimate(band_to_samples, smoothing=0)
    mu = estimate_mu(band_to_samples)
    var = estimate_var(band_to_samples, mu)
    
    potato_counts = {}

    for weight in count_df:
        posterior = get_posterior(weight, band_to_samples, pi, mu, var)
        for band in posterior:
            if band in potato_counts:
                potato_counts[band]+= posterior[band]
            else:
                potato_counts[band] = posterior[band]
    return potato_counts

In [32]:
pseudo_v_w = get_pseudo_counts(band_to_samples_venezia_w, df_venezia['TuberWeight'])
pseudo_v_cr = get_pseudo_counts(band_to_samples_venezia_cr, df_venezia['CubeRoot'])

print('True Distribution : \n', dict(true_dist_venezia))
print()
print('Estimated distribution based on TuborWeight :\n', pseudo_v_w)
print('KL Divergence of categorical distribution :\n', calculate_KL(pseudo_v_w, true_dist_venezia))
#print('W Distance of categorical distribution :\n', calculate_W(pseudo_v_w, true_dist_venezia))
print()
print('Estimated distribution based on CubeRoot :\n', pseudo_v_cr)
print('KL Divergence of categorical distribution :\n', calculate_KL(pseudo_v_cr, true_dist_venezia))
#print('W Distance of categorical distribution :\n', calculate_W(pseudo_v_cr, true_dist_venezia))

True Distribution : 
 {32.5: 54, 17.5: 28, 37.5: 63, 22.5: 59, 42.5: 25, 27.5: 72, 47.5: 2}

Estimated distribution based on TuborWeight :
 {32.5: 53.27822655718223, 17.5: 29.752785989336033, 37.5: 66.8024110577931, 22.5: 57.27567820007481, 42.5: 21.45108994420267, 27.5: 71.15077886331413, 47.5: 3.289029388097007}
KL Divergence of categorical distribution :
 0.0025457348122189826

Estimated distribution based on CubeRoot :
 {32.5: 53.69308723516428, 17.5: 29.155539016212213, 37.5: 69.17044249708555, 22.5: 58.842572419724874, 42.5: 20.589883899203837, 27.5: 68.22898578269398, 47.5: 3.319489149915194}
KL Divergence of categorical distribution :
 0.003823010347334399


In [33]:
for x in pseudo_v_w:
    print(round(pseudo_v_cr[x],2))
    
    

53.69
29.16
69.17
58.84
20.59
68.23
3.32


In [34]:
pseudo_s_w = get_pseudo_counts(band_to_samples_soraya_w, df_soraya['TuberWeight'])
pseudo_s_cr = get_pseudo_counts(band_to_samples_soraya_cr, df_soraya['CubeRoot'])

print('True Distribution : \n', dict(true_dist_soraya))
print()
print('Estimated distribution based on TuborWeight :\n', pseudo_s_w)
print('KL Divergence of categorical distribution :\n', calculate_KL(pseudo_s_w, true_dist_soraya))
#print('W Distance of categorical distribution :\n', calculate_W(pseudo_s_w, true_dist_soraya))

print()
print('Estimated distribution based on CubeRoot :\n', pseudo_s_cr)
print('KL Divergence of categorical distribution :\n', calculate_KL(pseudo_s_cr, true_dist_soraya))
#print('W Distance of categorical distribution :\n', calculate_W(pseudo_s_cr, true_dist_soraya))


True Distribution : 
 {32.5: 3, 17.5: 11, 67.5: 1, 52.5: 46, 37.5: 13, 22.5: 3, 57.5: 22, 42.5: 16, 27.5: 2, 62.5: 11, 47.5: 37}

Estimated distribution based on TuborWeight :
 {32.5: 4.938587989687508, 17.5: 11.14882721636118, 67.5: 0.9923182759638545, 52.5: 48.070915556681435, 37.5: 11.28060894987548, 22.5: 2.851461393308298, 57.5: 19.281845622973414, 42.5: 17.14134542837935, 27.5: 1.999641699112065, 62.5: 13.752084886211712, 47.5: 33.542362981445685}
KL Divergence of categorical distribution :
 0.007929621923797913

Estimated distribution based on CubeRoot :
 {32.5: 3.3307561970876027, 17.5: 9.5630230779995, 67.5: 1.3430026613756625, 52.5: 47.47415393837573, 37.5: 14.121155306557219, 22.5: 4.387697622027692, 57.5: 23.97257852229002, 42.5: 15.092538630524619, 27.5: 2.008791963417285, 62.5: 9.083104867577914, 47.5: 34.623197212766776}
KL Divergence of categorical distribution :
 0.005232735236187921


9.34
4.64
2.02
3.92
13.24
15.05
41.88
35.98
25.63
12.24
1.06
